In [33]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [34]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool


class Agents:
    def pathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside {file_path} path",
            backstory="You are fluent in Korean, and you are very good at finding files.",
            allow_delegation=False,
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )
    
    def fileSelector(self):
        return Agent(
            role="fileSelector",
            goal="Search One file that fits {question}, and Make one file that should be collected the paths of all files associated with the file",
            backstory="You are a file search expert and fluent in Korean. You have a great ability to read and analyze the details of the file.",
            allow_delegation=False,
            verbose=True,
            tools=[
                FileReadTool(),
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Finds and summarizes the markdown files that can solve the {question}",
            backstory="You are fluent in Korean, and you are very good at finding and summarizing files. Summarized information is essential for problem solving",
            verbose=True,
            tools=[
                FileReadTool(),
            ],
            max_iter=10,
        )

In [39]:
from crewai import Task


class Tasks:
    def pathSearch(self, agent):
        return Task(
            description="Finds the markdown files inside {file_path} path",
            expected_output="Your final answer MUST relative path",
            agent=agent,
            output_file="MarkdownPath.md",
        )

    def fileSelct(slef, agent, context):
        return Task(
            description="Based on the pathSearch, Search Only one file that can solve {question}. There are other documents linked by the symbol '[[...]]' in that file. Find all of the '[[...]]'s and print out the ONLY file path associated with the word in it in pathSearch. All file paths should EXIST in that pathSearch Output. If the relevant file does not exist, Just Say 'NOTHING'.",
            expected_output="""
            Your final answer MUST include the path of the first file and the path of other files within that file.
            It doesn't include ANYTHING other than file paths. 
            Example Answer 1
            Main file
            ./Algorithm/Algorithm Content/Tree/MST(Minimum Spanning Tree).md
            Related Files
            ./Algorithm/Algorithm Content/Graph Theory/DFS(Depth-First Search).md
            ./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md
            ./Algorithm/Algorithm Content/Tree/Union Find.md

            Example Answer 2
            Main file
            ./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md
            Related Files
            NOTHING

            Example Answer 3
            Main file
            ./Algorithm/a.md
            Related Files
            ./Algorithm/b.md
            ./Algorithm/c.md
            ./Algorithm/d.md
            ./Algorithm/e.md
            """,
            agent=agent,
            context=context,
            output_file="associateFilePath.md",
        )

    def research(self, agent, context):
        return Task(
            description="Based on the pathSearch, Gather and analyze the most relevant markdown files for troubleshooting {question}. Provide a summary of markdown files and what is an important point in files. If there is a symbol '[[file_name]]' in the markdown file, you should also look into the markdown file with the file_name.",
            expected_output="Your final answer MUST be a detailed summary of the markdown files in Korean. Include Path to all imported files, why the file was imported, what is an important point and If there is an example for {question}, include it. If you can't find the file, JUST Answer that you don't know",
            agent=agent,
            context=context,
            output_file="questionAnswer.md",
        )

In [38]:
from crewai import Crew

agent = Agents()
tasks = Tasks()

pathSearcher = agent.pathSearcher()
fileSelector = agent.fileSelector()
researcher = agent.researcher()

pathSearcher_task = tasks.pathSearch(pathSearcher)
fileSelector_task = tasks.fileSelct(fileSelector, [pathSearcher_task])
researcher_task = tasks.research(researcher, [pathSearcher_task])

crew = Crew(
    agents=[
        pathSearcher,
        fileSelector,
    ],
    tasks=[
        pathSearcher_task,
        fileSelector_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    dict(
        file_path="./Algorithm/Algorithm Content",
        question="Fenwick Tree가 뭐야?",
    )
)

2025-01-06 21:07:01,763 - 21640 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: pathSearcher
 [INFO]: == Starting Task: Finds the markdown files inside ./Algorithm/Algorithm Content path


> Entering new CrewAgentExecutor chain...
I need to find markdown files inside the specified directory `./Algorithm/Algorithm Content`. To start, I will list the files in that directory to see what I can find.

Action: List files in directory  
Action Input: {"directory": "./Algorithm/Algorithm Content"}  
 

File paths: 
-./Algorithm/Algorithm Content/Array\Binary Search.md
- ./Algorithm/Algorithm Content/Array\MITM(Meet in the middle).md
- ./Algorithm/Algorithm Content/Array\PBS(Parallel Binary Search).md
- ./Algorithm/Algorithm Content/Graph Theory\Articulation Points And Bridges.md
- ./Algorithm/Algorithm Content/Graph Theory\BFS(Breadth-First Search).md
- ./Algorithm/Algorithm Content/Graph Theory\CCW(Counter Clock Wise).md
- ./Algorithm/Algorithm Content/Graph Theory\DFS(Depth-First Search).md
- ./Algorithm/Algorithm Content/Graph Theory\Dijkstr